# Document Classification with Naive Bayes - Lab

## Introduction

In this lesson, you'll practice implementing the Naive Bayes algorithm on your own.

## Objectives

In this lab you will:  

* Implement document classification using Naive Bayes

## Import the dataset

To start, import the dataset stored in the text file `'SMSSpamCollection'`.

In [13]:
# Your code here
import pandas as pd

df = pd.read_csv('SMSSpamCollection', sep='\t', header=None)
df.columns = ['type', 'text']

In [14]:
df

,type,text
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will ü b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


## Account for class imbalance

To help your algorithm perform more accurately, subset the dataset so that the two classes are of equal size. To do this, keep all of the instances of the minority class (spam) and subset examples of the majority class (ham) to an equal number of examples.

In [23]:
# Your code here
df['type'].value_counts()

df_sub_ham = df[df.type=='ham'].sample(len(df[df.type=='spam']))
df[df['type']=='spam']

,type,text
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
5,spam,FreeMsg Hey there darling it's been 3 week's n...
8,spam,WINNER!! As a valued network customer you have...
9,spam,Had your mobile 11 months or more? U R entitle...
11,spam,"SIX chances to win CASH! From 100 to 20,000 po..."
...,...,...
5537,spam,Want explicit SEX in 30 secs? Ring 02073162414...
5540,spam,ASKED 3MOBILE IF 0870 CHATLINES INCLU IN FREE ...
5547,spam,Had your contract mobile 11 Mnths? Latest Moto...
5566,spam,REMINDER FROM O2: To get 2.50 pounds free call...


In [24]:
df_sub = pd.concat([df[df['type']=='spam'], df_sub_ham], axis=0)

In [26]:
df_sub['type'].value_counts()

spam    747
ham     747
Name: type, dtype: int64

In [49]:
p_classes = dict(df_sub['type'].value_counts(normalize=True))

## Train-test split

Now implement a train-test split on the dataset: 

In [31]:
# Your code here
from sklearn.model_selection import train_test_split
X = df_sub['text']
y = df_sub['type']
X_train, X_test, y_train, y_test = train_test_split(X,y)
train_df = pd.concat([X_train, y_train], axis=1)
test_df = pd.concat([X_test, y_test], axis=1)
train_df

,text,type
4898,I cant pick the phone right now. Pls send a me...,ham
5018,Dear 0776xxxxxxx U've been invited to XCHAT. T...,spam
1767,SMS AUCTION You have won a Nokia 7250i. This i...,spam
2737,Really? I crashed out cuddled on my sofa.,ham
2514,U have won a nokia 6230 plus a free digital ca...,spam
...,...,...
192,I'm sorry. I've joined the league of people th...,ham
955,Filthy stories and GIRLS waiting for your,spam
4746,Camera - You are awarded a SiPix Digital Camer...,spam
4474,S but not able to sleep.,ham


## Create the word frequency dictionary for each class

Create a word frequency dictionary for each class: 

In [37]:
# Your code here
class_word_freq = {}
classes = train_df['type'].unique()
classes

array(['ham', 'spam'], dtype=object)

In [39]:
for class_ in classes:
#     print(class_)
    temp_df = train_df[train_df['type']==class_]
    bag = {}
    for idx in temp_df.index:
        doc = train_df['text'][idx]
        for word in doc.split():
            bag[word] = bag.get(word, 0) + 1
#     print(bag)
    class_word_freq[class_] = bag

## Count the total corpus words
Calculate V, the total number of words in the corpus: 

In [40]:
# Your code here
vocabs = set()
for text in train_df['text']:
    for word in text.split():
        vocabs.add(word)
V = len(vocabs)
V

5802

## Create a bag of words function

Before implementing the entire Naive Bayes algorithm, create a helper function `bag_it()` to create a bag of words representation from a document's text.

In [41]:
# Your code here
def bag_it(doc):
    bag = {}
    for word in doc.split():
        bag[word] = bag.get(word, 0) + 1
    return bag
    

## Implementing Naive Bayes

Now, implement a master function to build a naive Bayes classifier. Be sure to use the logarithmic probabilities to avoid underflow.

In [43]:
class_word_freq.keys()

dict_keys(['ham', 'spam'])

In [52]:
# Your code here
def classify_doc(doc, class_word_freq, p_classes, V, return_posteriors=False):
    bag = bag_it(doc)
    classes = []
    posteriors = []
    
    for class_ in class_word_freq.keys():
        p = np.log(p_classes[class_])
        for word in bag.keys():
            num = bag[word]
            den = class_word_freq[class_].get(word, 0) + V
            p *= np.log(num/den)
        classes.append(class_)
        posteriors.append(p)
    
    if return_posteriors:
        print(posteriors)
    return classes[np.argmax(posteriors)]

## Test your classifier

Finally, test your classifier and measure its accuracy. Don't be perturbed if your results are sub-par; industry use cases would require substantial additional preprocessing before implementing the algorithm in practice.

In [53]:
# Your code here
import numpy as np
y_hat_train = X_train.map(lambda x: classify_doc(x, class_word_freq, p_classes, V))

In [55]:
train_resid = y_hat_train == y_train
train_accuracy = sum(train_resid)/len(train_resid)

print('Training Accuracy: ', train_accuracy)

Training Accuracy:  0.50625


## Level up (Optional)

Rework your code into an appropriate class structure so that you could easily implement the algorithm on any given dataset.

## Summary

Well done! In this lab, you practiced implementing Naive Bayes for document classification!